# SAR based Recommender

## SAR algorithm

The following figure presents a high-level architecture of SAR. 

At a very high level, two intermediate matrices are created and used to generate a set of recommendation scores:

- An item similarity matrix $S$ estimates item-item relationships.
- An affinity matrix $A$ estimates user-item relationships.

Recommendation scores are then created by computing the matrix multiplication $A\times S$.

Optional steps (e.g. "time decay" and "remove seen items") are described in the details below.

<img src="https://recodatasets.blob.core.windows.net/images/sar_schema.svg?sanitize=true">

### Compute item co-occurrence and item similarity

SAR defines similarity based on item-to-item co-occurrence data. Co-occurrence is defined as the number of times two items appear together for a given user. We can represent the co-occurrence of all items as a $m\times m$ matrix $C$, where $c_{i,j}$ is the number of times item $i$ occurred with item $j$, and $m$ is the total number of items.

The co-occurence matric $C$ has the following properties:

- It is symmetric, so $c_{i,j} = c_{j,i}$
- It is nonnegative: $c_{i,j} \geq 0$
- The occurrences are at least as large as the co-occurrences. I.e., the largest element for each row (and column) is on the main diagonal: $\forall(i,j) C_{i,i},C_{j,j} \geq C_{i,j}$.

Once we have a co-occurrence matrix, an item similarity matrix $S$ can be obtained by rescaling the co-occurrences according to a given metric. Options for the metric include `Jaccard`, `lift`, and `counts` (meaning no rescaling).


If $c_{ii}$ and $c_{jj}$ are the $i$th and $j$th diagonal elements of $C$, the rescaling options are:

- `Jaccard`: $s_{ij}=\frac{c_{ij}}{(c_{ii}+c_{jj}-c_{ij})}$
- `lift`: $s_{ij}=\frac{c_{ij}}{(c_{ii} \times c_{jj})}$
- `counts`: $s_{ij}=c_{ij}$

In general, using `counts` as a similarity metric favours predictability, meaning that the most popular items will be recommended most of the time. `lift` by contrast favours discoverability/serendipity: an item that is less popular overall but highly favoured by a small subset of users is more likely to be recommended. `Jaccard` is a compromise between the two.


### Compute user affinity scores

The affinity matrix in SAR captures the strength of the relationship between each individual user and the items that user has already interacted with. SAR incorporates two factors that can impact users' affinities: 

- It can consider information about the **type** of user-item interaction through differential weighting of different events (e.g. it may weigh events in which a user rated a particular item more heavily than events in which a user viewed the item).
- It can consider information about **when** a user-item event occurred (e.g. it may discount the value of events that take place in the distant past.

Formalizing these factors produces us an expression for user-item affinity:

$$a_{ij}=\sum_k w_k \left(\frac{1}{2}\right)^{\frac{t_0-t_k}{T}} $$

where the affinity $a_{ij}$ for user $i$ and item $j$ is the weighted sum of all $k$ events involving user $i$ and item $j$. $w_k$ represents the weight of a particular event, and the power of 2 term reflects the temporally-discounted event. The $(\frac{1}{2})^n$ scaling factor causes the parameter $T$ to serve as a half-life: events $T$ units before $t_0$ will be given half the weight as those taking place at $t_0$.

Repeating this computation for all $n$ users and $m$ items results in an $n\times m$ matrix $A$. Simplifications of the above expression can be obtained by setting all the weights equal to 1 (effectively ignoring event types), or by setting the half-life parameter $T$ to infinity (ignoring transaction times).

### Remove seen item

Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

### Top-k item calculation

The personalized recommendations for a set of users can then be obtained by multiplying the affinity matrix ($A$) by the similarity matrix ($S$). The result is a recommendation score matrix, where each row corresponds to a user, each column corresponds to an item, and each entry corresponds to a user / item pair. Higher scores correspond to more strongly recommended items.

It is worth noting that the complexity of recommending operation depends on the data size. SAR algorithm itself has $O(n^3)$ complexity. Therefore the single-node implementation is not supposed to handle large dataset in a scalable manner. Whenever one uses the algorithm, it is recommended to run with sufficiently large memory. 

In [3]:
import itertools
import logging
import os
import gc

import numpy as np
import pandas as pd
import papermill as pm
from sklearn.externals import joblib 

from python_splitters import python_stratified_split
from python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from sar_singlenode import SARSingleNode

### Load Data

In [4]:
# Constants to be used
TOP_K = 10
USER, ITEM, RATING, TIME, PRED = '회원번호', 'ISBN', '평점', '일자', 'SCORE'  # 카드 데이터에 맞게 수정해야 함 !

In [5]:
# 이미 데이터가 나누어져 있으면 이 부분 Skip !\
data = pd.read_csv('book_transactions.csv', encoding='cp949').dropna(axis=0).query('ISBN != "-"')

# Add ratings column
data[RATING] = 1

data = data.loc[:,[USER,ITEM,RATING,TIME]]
data

,회원번호,ISBN,평점,일자
0,292,9788964166178,1,20140621
1,292,9788966868674,1,20140621
2,292,9788937898334,1,20140621
3,294,9788926321379,1,20140528
4,294,9788925292564,1,20140528
5,294,9788925288239,1,20140528
6,299,9788996287001,1,20140519
7,299,9788966942909,1,20140519
8,300,9788936456221,1,20140518
11,308,9788992751193,1,20140512


### Split Data

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because SAR generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.


In [6]:
# 반드시 실행해야 함!
header = {
    "col_user": USER,
    "col_item": ITEM,
    "col_rating": RATING,
    "col_timestamp": TIME,
    "col_prediction": PRED,
}

In [7]:
# 이미 데이터가 나누어져 있으면 이 부분 Skip하고, 나누어진 데이터를 train과 test 이름의 변수에 저장!
train, test = python_stratified_split(data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42)

In [8]:
# 이미 데이터가 나누어져 있으면 이 부분 Skip !
joblib.dump([data, train, test], 'sar_data.pkl')

del data; gc.collect()

7

### Build SAR Model

In [9]:
# 이 부분부터 아래 끝까지 실행해야 함!

# set log level to INFO
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SARSingleNode(
    similarity_type='cooccurrence',   # 'jaccard', 'lift', 'cooccurrence' 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True, 
#    normalize=True,
    **header
)

In [10]:
model.fit(train)
#joblib.dump(model, 'sar_model.pkl')

2019-11-03 20:27:40,683 INFO     Collecting user affinity matrix
2019-11-03 20:27:40,694 INFO     Calculating time-decayed affinities
2019-11-03 20:27:40,872 INFO     Creating index columns
2019-11-03 20:27:40,998 INFO     Building user affinity sparse matrix
2019-11-03 20:27:41,011 INFO     Calculating item co-occurrence
2019-11-03 20:27:41,253 INFO     Calculating item similarity
2019-11-03 20:27:41,254 INFO     Using co-occurrence based similarity
2019-11-03 20:27:41,254 INFO     Done training


### Make Recommendations

In [11]:
top_k = model.recommend_k_items(test, remove_seen=True)
#joblib.dump(top_k, 'sar_top_k.pkl')
top_k.head(10)

2019-11-03 20:27:49,654 INFO     Calculating recommendation scores
2019-11-03 20:27:52,952 INFO     Removing seen items


,회원번호,ISBN,SCORE
0,292,9788928310258,1.999995
1,292,9788966866946,0.999998
2,292,9788926975169,0.999998
3,292,9788961332637,0.999998
4,292,9788905037642,0.999998
5,292,9788960275348,0.999998
6,292,9791185419022,0.999998
7,292,9788967904838,0.999998
8,292,9791185507958,0.999998
9,292,9788905038984,0.999998


### Evaluate SAR Model

In [12]:
args = [test, top_k]
kwargs = dict(col_user=USER, 
              col_item=ITEM, 
              col_rating=RATING, 
              col_prediction=PRED, 
              relevancy_method='top_k', 
              k=TOP_K)

#eval_map = map_at_k(*args, **kwargs)
#eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

print(f"Model:",
      f"Top K:\t\t {TOP_K}",
#      f"MAP:\t\t {eval_map:f}",
#      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')

Model:
Top K:		 10
Precision@K:	 0.041672
Recall@K:	 0.153619


### 사용자 편향점수와 아이템 편향점수를 고려하여 score를 조정

In [13]:
# 가맹점별로 가망고객을 예측하려면, 사용자와 가맹점 편향점수를 고려할 필요가 있음!
top_k_adj = top_k.copy()
top_k_adj = pd.merge(top_k_adj, top_k_adj.groupby(USER)[PRED].agg([('USER_MEAN', np.mean)]).reset_index(), on=USER)
top_k_adj = pd.merge(top_k_adj, top_k_adj.groupby(ITEM)[PRED].agg([('ITEM_MEAN', np.mean)]).reset_index(), on=ITEM)
top_k_adj['ADJ_SCORE'] = top_k_adj['USER_MEAN'] + top_k_adj['ITEM_MEAN'] - top_k_adj[PRED].mean()
top_k_adj = top_k_adj.iloc[:,[0,1,2,5]].sort_values(by=USER)
top_k_adj

,회원번호,ISBN,SCORE,ADJ_SCORE
0,292,9788928310258,1.999995,-24.354527
243,292,9788961332637,0.999998,-31.575232
259,292,9788905037642,0.999998,-34.387447
261,292,9788960275348,0.999998,-25.020255
405,292,9791185419022,0.999998,-24.388259
469,292,9788967904838,0.999998,-33.887487
471,292,9791185507958,0.999998,-34.387408
472,292,9788905038984,0.999998,-34.387408
239,292,9788926975169,0.999998,-33.887537
236,292,9788966866946,0.999998,-33.720815


# End